# Recurrent Neural Network


**Notasi**:
- Superscript $[l]$ menunjukkan objek yang terkait dengan lapisan $l^{th}$.

- Superscript $(i)$ menunjukkan objek yang terkait dengan data ke $i^{th}$.

- Superscript $\langle t \rangle$ menunjukkan suatu objek pada langkah $t^{th}$.
    
- Subscript $i$ menunjukkan entri $i^{th}$ dari sebuah vektor.

**Contoh**:
- $a^{(2)[3]<4>}_5$ menunjukkan aktivasi contoh pelatihan ke-2 (2), lapisan ke-3 [3], langkah ke-4 <4>, dan entri ke-5 dalam vektor.

<a name='0'></a>
## Packages

In [1]:
import numpy as np
from rnn_utils import *
from public_tests import *

<a name='1'></a>
## Propagasi Forward untuk Recurrent Neural Network Dasar

RNN dasar yang akan Anda terapkan memiliki struktur berikut:

Dalam contoh ini, $T_x = T_y$.

<img src="images/RNN.png" style="width:500;height:300px;">
<caption><center><font color='purple'><b>Gambar 1</b>: RNN model </center></caption>

### Dimensi input $x$

#### Input dengan jumlah unit $n_x$
* Untuk sekali langkah dari satu data input, $x^{(i) \langle t \rangle }$ adalah vektor input satu dimensi
* Dengan menggunakan bahasa sebagai contoh, bahasa dengan vocabulary 5000 kata dapat diberlakukan encoding secara one-hot ke dalam vektor yang memiliki 5000 unit. Jadi $x^{(i)\langle t \rangle}$ akan berbentuk (5000,)
* Notasi $n_x$ digunakan di sini untuk menunjukkan jumlah unit dalam satu langkah dari satu data latih

#### Langkah pindah berukuran $T_{x}$
* Recurrent neural network memiliki beberapa langkah, yang akan Anda indeks dengan $t$.
* Dalam pelajaran, Anda melihat satu data latih $x^{(i)}$ yang terdiri dari beberapa langkah pindah $T_x$. Di notebook ini, $T_{x}$ akan menunjukkan jumlah langkah pindah dalam urutan terpanjang.

#### Batch ukuran $m$
* Katakanlah kita memiliki mini-batches, masing-masing berisi 20 data latih
* Untuk mendapatkan manfaat dari vektorisasi, Anda akan menumpuk 20 kolom data $x^{(i)}$
* Misalnya tensor ini berbentuk (5000,20,10)
* Anda akan menggunakan $m$ untuk menunjukkan jumlah data latih
* Jadi, bentuk mini-batchnya adalah $(n_x,m,T_x)$

#### Tensor 3D berbentuk $(n_{x},m,T_{x})$
*Tensor $x$ 3 dimensi berbentuk $(n_x,m,T_x)$ mewakili input $x$ yang dimasukkan ke RNN

#### Mengambil potongan 2D untuk setiap langkah pindah: $x^{\langle t \rangle}$
* Pada setiap langkah pindah, Anda akan menggunakan mini-batch data latih (bukan hanya satu contoh)
* Jadi, untuk setiap langkah pindah $t$, Anda akan menggunakan potongan 2D bentuk $(n_x,m)$
* Irisan 2D ini disebut sebagai $x^{\langle t \rangle}$. Nama variabel dalam code adalah `xt`.

### Definisi hidden state $a$

* Aktivasi $a^{\langle t \rangle}$ yang diteruskan ke RNN dari satu langkah pindah ke langkah lainnya disebut "hidden state".

### Dimensi hidden state $a$

* Mirip dengan input tensor $x$, hidden state untuk data latih tunggal adalah vektor dengan panjang $n_{a}$
* Jika Anda menyertakan mini-batch dari data latih berukuran $m$, bentuk mini-batch adalah $(n_{a},m)$
* Saat Anda memasukkan dimensi langkah pindah, bentuk hidden statenya adalah $(n_{a}, m, T_x)$
* Anda akan mengulangi langkah-langkah waktu dengan indeks $t$, dan bekerja dengan potongan 2D dari tensor 3D
* Irisan 2D ini disebut sebagai $a^{\langle t \rangle}$
* Dalam code, nama variabel yang digunakan adalah `a_prev` atau `a_next`, bergantung pada fungsi yang diimplementasikan
* Bentuk irisan 2D ini adalah $(n_{a}, m)$

### Dimensi prediksi $\hat{y}$
* Mirip dengan input dan hidden state, $\hat{y}$ adalah tensor 3D berbentuk $(n_{y}, m, T_{y})$
* $n_{y}$: jumlah unit dalam vektor yang mewakili prediksi
* $m$: jumlah data dalam mini-batch
* $T_{y}$: jumlah langkah pindah dalam prediksi
* Untuk satu langkah pindah $t$, irisan 2D $\hat{y}^{\langle t \rangle}$ memiliki bentuk $(n_{y}, m)$
* Di dalam code, nama variabelnya adalah:
    - `y_pred`: $\hat{y}$
    - `yt_pred`: $\hat{y}^{\langle t \rangle}$

Cara mengimplementasikan RNN:

### Langkah:
1. implementasi penghitungan yang diperlukan untuk satu langkah pindah RNN.
2. implementasi loop untuk setiap langkah pindah $T_x$ untuk memproses semua input, satu per satu.

<a name='1-1'></a>
### RNN cell

Anda dapat menganggap recurrent neural network sebagai penggunaan berulang dari satu sel. Pertama, Anda akan menerapkan komputasi untuk satu langkah pindah. Gambar berikut menjelaskan operasi untuk satu langkah waktu sel RNN:

<img src="images/rnn_step_forward_figure2_v3a.png" style="width:700px;height:300px;">
<caption><center><font color='purple'><b>Gambar 2</b>: Sel RNN dasar. Mengambil input $x^{\langle t \rangle}$ (input saat ini) dan $a^{\langle t - 1\rangle}$ (hidden state sebelumnya yang berisi informasi sebelumnya), dan menghasilkan output $a^{\langle t \rangle}$ yang diberikan ke sel RNN berikutnya dan juga digunakan untuk memprediksi $\hat{y}^{\langle t \rangle}$
</center></caption>

**`RNN cell` versus `RNN_cell_forward`**:
* Perhatikan bahwa sel RNN mengeluarkan hidden state $a^{\langle t \rangle}$.
* `RNN cell` ditampilkan pada gambar sebagai kotak sisi dalam dengan garis padat
* Fungsi yang akan Anda implementasikan, `rnn_cell_forward`, juga menghitung prediksi $\hat{y}^{\langle t \rangle}$
* `RNN_cel_forward` ditampilkan pada gambar sebagai kotak sisi luar dengan garis putus-putus

<a name='ex-1'></a>
### Tugas 1 - rnn_cell_forward

Implementasikan sel RNN yang dijelaskan pada Gambar.

**Petunjuk**:
1. Hitung hidden state dengan aktivasi tanh: $a^{\langle t \rangle} = \tanh(W_{aa} a^{\langle t-1 \rangle} + W_{ax} x^{\langle t \rangle} + b_a)$
2. Dengan menggunakan $a^{\langle t \rangle}$ hidden state baru, hitung prediksi $\hat{y}^{\langle t \rangle} = softmax(W_{ya} a^{\langle t \rangle} + b_y)$. (Fungsi `softmax` disediakan)
3. Simpan $(a^{\langle t \rangle}, a^{\langle t-1 \rangle}, x^{\langle t \rangle}, parameters)$ dalam `cache`
4. Return $a^{\langle t \rangle}$ , $\hat{y}^{\langle t \rangle}$ dan `cache`

#### Petunjuk Tambahan
* Sedikit informasi lebih lanjut tentang [numpy.tanh](https://numpy.org/devdocs/reference/generated/numpy.tanh.html)
* Dalam tugas ini, terdapat fungsi `softmax` yang dapat Anda gunakan. Itu terletak di file 'rnn_utils.py' dan telah diimpor.
* Untuk perkalian matriks, gunakan [numpy.dot](https://docs.scipy.org/doc/numpy/reference/generated/numpy.dot.html)

In [2]:
# UNQ_C1 (UNIQUE CELL IDENTIFIER, DO NOT EDIT)
# GRADED FUNCTION: rnn_cell_forward

def rnn_cell_forward(xt, a_prev, parameters):
    """
    Implements a single forward step of the RNN-cell as described in Figure (2)

    Arguments:
    xt -- your input data at timestep "t", numpy array of shape (n_x, m).
    a_prev -- Hidden state at timestep "t-1", numpy array of shape (n_a, m)
    parameters -- python dictionary containing:
                        Wax -- Weight matrix multiplying the input, numpy array of shape (n_a, n_x)
                        Waa -- Weight matrix multiplying the hidden state, numpy array of shape (n_a, n_a)
                        Wya -- Weight matrix relating the hidden-state to the output, numpy array of shape (n_y, n_a)
                        ba --  Bias, numpy array of shape (n_a, 1)
                        by -- Bias relating the hidden-state to the output, numpy array of shape (n_y, 1)
    Returns:
    a_next -- next hidden state, of shape (n_a, m)
    yt_pred -- prediction at timestep "t", numpy array of shape (n_y, m)
    cache -- tuple of values needed for the backward pass, contains (a_next, a_prev, xt, parameters)
    """
    
    # Retrieve parameters from "parameters"
    Wax = parameters["Wax"]
    Waa = parameters["Waa"]
    Wya = parameters["Wya"]
    ba = parameters["ba"]
    by = parameters["by"]
    
    ### START CODE HERE ### (≈2 lines)
    # compute next activation state using the formula given above
    a_next = np.tanh(np.dot(Waa, a_prev) + np.dot(Wax, xt) + ba)
    # compute output of the current cell using the formula given above
    yt_pred = softmax(np.dot(Wya, a_next) + by)
    ### END CODE HERE ###
    
    # store values you need for backward propagation in cache
    cache = (a_next, a_prev, xt, parameters)
    
    return a_next, yt_pred, cache

In [3]:
np.random.seed(1)
xt_tmp = np.random.randn(3, 10)
a_prev_tmp = np.random.randn(5, 10)
parameters_tmp = {}
parameters_tmp['Waa'] = np.random.randn(5, 5)
parameters_tmp['Wax'] = np.random.randn(5, 3)
parameters_tmp['Wya'] = np.random.randn(2, 5)
parameters_tmp['ba'] = np.random.randn(5, 1)
parameters_tmp['by'] = np.random.randn(2, 1)

a_next_tmp, yt_pred_tmp, cache_tmp = rnn_cell_forward(xt_tmp, a_prev_tmp, parameters_tmp)
print("a_next[4] = \n", a_next_tmp[4])
print("a_next.shape = \n", a_next_tmp.shape)
print("yt_pred[1] =\n", yt_pred_tmp[1])
print("yt_pred.shape = \n", yt_pred_tmp.shape)

# UNIT TESTS
rnn_cell_forward_tests(rnn_cell_forward)

a_next[4] = 
 [ 0.59584544  0.18141802  0.61311866  0.99808218  0.85016201  0.99980978
 -0.18887155  0.99815551  0.6531151   0.82872037]
a_next.shape = 
 (5, 10)
yt_pred[1] =
 [0.9888161  0.01682021 0.21140899 0.36817467 0.98988387 0.88945212
 0.36920224 0.9966312  0.9982559  0.17746526]
yt_pred.shape = 
 (2, 10)
All tests passed


**Expected Output**: 
```Python
a_next[4] = 
 [ 0.59584544  0.18141802  0.61311866  0.99808218  0.85016201  0.99980978
 -0.18887155  0.99815551  0.6531151   0.82872037]
a_next.shape = 
 (5, 10)
yt_pred[1] =
 [ 0.9888161   0.01682021  0.21140899  0.36817467  0.98988387  0.88945212
  0.36920224  0.9966312   0.9982559   0.17746526]
yt_pred.shape = 
 (2, 10)

```

<a name='1-2'></a>
### RNN Forward Pass

- Recurrent Neural Network (RNN) adalah repitisi sel RNN yang baru saja Anda buat.
    - Jika urutan input data Anda panjangnya 10 langkah pindah, maka Anda akan menggunakan kembali sel RNN sebanyak 10 kali
- Setiap sel mengambil dua input pada setiap langkah pindah:
    - $a^{\langle t-1 \rangle}$: Hidden state dari sel sebelumnya
    - $x^{\langle t \rangle}$: Data input langkah pindah saat ini
- Ini memiliki dua output pada setiap langkah waktu:
    - Hidden state ($a^{\langle t \rangle}$)
    - Prediksi ($y^{\langle t \rangle}$)
- Bobot dan bias $(W_{aa}, b_{a}, W_{ax}, b_{x})$ digunakan kembali setiap kali langkah pindah
    - Mereka dipertahankan di antara panggilan ke `rnn_cell_forward` dalam dictionary 'parameter'


<img src="images/rnn_forward_sequence_figure3_v3a.png" style="width:800px;height:180px;">
<caption><center><font color='purple'><b>Gambar 3</b>: RNN Dasar. Urutan input $x = (x^{\langle 1 \rangle}, x^{\langle 2 \rangle}, ..., x^{\langle T_x \rangle})$ dibawa melalui langkah pindah $T_x$. Network mengeluarkan $y = (y^{\langle 1 \rangle}, y^{\langle 2 \rangle}, ..., y^{\langle T_x \rangle})$. </center></caption>

<a name='ex-2'></a>
### Tugas 2 - rnn_forward

Menerapkan propagasi RNN forward yang dijelaskan pada Gambar 3.

**Petunjuk**:
* Buat array 3D nol, $a$ berbentuk $(n_{a}, m, T_{x})$ yang akan menyimpan semua hidden state yang dihitung oleh RNN
* Buat array 3D nol, $\hat{y}$, bentuk $(n_{y}, m, T_{x})$ yang akan menyimpan prediksi
    - Perhatikan bahwa dalam kasus ini, $T_{y} = T_{x}$ (prediksi dan input memiliki jumlah langkah pindah yang sama)
* Inisiasi hidden state 2D `a_next` dengan mengaturnya sama dengan hidden state awal, $a_{0}$
* Pada setiap langkah pindah $t$:
    - Dapatkan $x^{\langle t \rangle}$, yang merupakan potongan 2D dari $x$ untuk satu langkah pindah $t$
        - $x^{\langle t \rangle}$ memiliki bentuk $(n_{x}, m)$
        - $x$ memiliki bentuk $(n_{x}, m, T_{x})$
    - Perbarui hidden state 2D $a^{\langle t \rangle}$ (nama variabel `a_next`), prediksi $\hat{y}^{\langle t \rangle}$ dan cache dengan menjalankan `rnn_cell_forward`
        - $a^{\langle t \rangle}$ memiliki bentuk $(n_{a}, m)$
    - Simpan hidden state 2D di tensor 3D $a$, pada posisi $t^{th}$
        - $a$ memiliki bentuk $(n_{a}, m, T_{x})$
    - Simpan prediksi 2D $\hat{y}^{\langle t \rangle}$ (nama variabel `yt_pred`) di tensor 3D $\hat{y}_{pred}$ pada posisi $t^{th}$
        - $\hat{y}^{\langle t \rangle}$ memiliki bentuk $(n_{y}, m)$
        - $\hat{y}$ memiliki bentuk $(n_{y}, m, T_x)$
    - Tambahkan cache ke daftar cache
* Mengembalikan tensor 3D $a$ dan $\hat{y}$, serta daftar cache

#### Petunjuk Tambahan
- Beberapa dokumentasi bermanfaat di [np.zeros](https://docs.scipy.org/doc/numpy/reference/generated/numpy.zeros.html)
- Jika Anda memiliki array numpy 3 dimensi dan mengindeks berdasarkan dimensi ketiganya, Anda dapat menggunakan slicing array seperti ini: `var_name[:,:,i]`

In [4]:
# UNQ_C2 (UNIQUE CELL IDENTIFIER, DO NOT EDIT)
# GRADED FUNCTION: rnn_forward

def rnn_forward(x, a0, parameters):
    """
    Implement the forward propagation of the recurrent neural network described in Figure (3).

    Arguments:
    x -- Input data for every time-step, of shape (n_x, m, T_x).
    a0 -- Initial hidden state, of shape (n_a, m)
    parameters -- python dictionary containing:
                        Waa -- Weight matrix multiplying the hidden state, numpy array of shape (n_a, n_a)
                        Wax -- Weight matrix multiplying the input, numpy array of shape (n_a, n_x)
                        Wya -- Weight matrix relating the hidden-state to the output, numpy array of shape (n_y, n_a)
                        ba --  Bias numpy array of shape (n_a, 1)
                        by -- Bias relating the hidden-state to the output, numpy array of shape (n_y, 1)

    Returns:
    a -- Hidden states for every time-step, numpy array of shape (n_a, m, T_x)
    y_pred -- Predictions for every time-step, numpy array of shape (n_y, m, T_x)
    caches -- tuple of values needed for the backward pass, contains (list of caches, x)
    """
    
    # Initialize "caches" which will contain the list of all caches
    caches = []
    
    # Retrieve dimensions from shapes of x and parameters["Wya"]
    n_x, m, T_x = x.shape
    n_y, n_a = parameters["Wya"].shape
    
    ### START CODE HERE ###
    
    # initialize "a" and "y_pred" with zeros (≈2 lines)
    a = np.zeros((n_a, m, T_x))
    y_pred = np.zeros((n_y, m, T_x))
    
    # Initialize a_next (≈1 line)
    a_next = a0
    
    # loop over all time-steps
    for t in range(T_x):
        # Update next hidden state, compute the prediction, get the cache (≈1 line)
        a_next, yt_pred, cache = rnn_cell_forward(x[:, :, t], a_next, parameters)
        # Save the value of the new "next" hidden state in a (≈1 line)
        a[:,:,t] = a_next
        # Save the value of the prediction in y (≈1 line)
        y_pred[:,:,t] = yt_pred
        # Append "cache" to "caches" (≈1 line)
        caches.append(cache)
    ### END CODE HERE ###
    
    # store values needed for backward propagation in cache
    caches = (caches, x)
    
    return a, y_pred, caches

In [5]:
np.random.seed(1)
x_tmp = np.random.randn(3, 10, 4)
a0_tmp = np.random.randn(5, 10)
parameters_tmp = {}
parameters_tmp['Waa'] = np.random.randn(5, 5)
parameters_tmp['Wax'] = np.random.randn(5, 3)
parameters_tmp['Wya'] = np.random.randn(2, 5)
parameters_tmp['ba'] = np.random.randn(5, 1)
parameters_tmp['by'] = np.random.randn(2, 1)

a_tmp, y_pred_tmp, caches_tmp = rnn_forward(x_tmp, a0_tmp, parameters_tmp)
print("a[4][1] = \n", a_tmp[4][1])
print("a.shape = \n", a_tmp.shape)
print("y_pred[1][3] =\n", y_pred_tmp[1][3])
print("y_pred.shape = \n", y_pred_tmp.shape)
print("caches[1][1][3] =\n", caches_tmp[1][1][3])
print("len(caches) = \n", len(caches_tmp))

#UNIT TEST    
rnn_forward_test(rnn_forward)

a[4][1] = 
 [-0.99999375  0.77911235 -0.99861469 -0.99833267]
a.shape = 
 (5, 10, 4)
y_pred[1][3] =
 [0.79560373 0.86224861 0.11118257 0.81515947]
y_pred.shape = 
 (2, 10, 4)
caches[1][1][3] =
 [-1.1425182  -0.34934272 -0.20889423  0.58662319]
len(caches) = 
 2
All tests passed


**Expected Output**:

```Python
a[4][1] = 
 [-0.99999375  0.77911235 -0.99861469 -0.99833267]
a.shape = 
 (5, 10, 4)
y_pred[1][3] =
 [ 0.79560373  0.86224861  0.11118257  0.81515947]
y_pred.shape = 
 (2, 10, 4)
caches[1][1][3] =
 [-1.1425182  -0.34934272 -0.20889423  0.58662319]
len(caches) = 
 2
```

<a name='2'></a>
## Long Short-Term Memory (LSTM).

Gambar berikut menunjukkan operasi sel LSTM:

<img src="images/LSTM_figure4_v3a.png" style="width:500;height:400px;">
<caption><center><font color='purple'><b>Gambar 4</b>: sel LSTM. Ini melacak dan memperbarui "cell state", atau variabel memori $c^{\langle t \rangle}$ pada setiap langkah pindah, yang mungkin berbeda dari $a^{\langle t \rangle}$.
Catatan, $softmax^{}$ menyertakan dense layer dan softmax.</center></caption>

Mirip dengan contoh RNN di atas, Anda akan memulai dengan mengimplementasikan sel LSTM untuk satu langkah pindah. Kemudian, Anda akan memanggilnya secara iteratif dari dalam "for loop" agar ia memproses input dengan langkah pindah $T_x$.

### Overview dari gates dan states

#### Forget Gate $\mathbf{\Gamma}_{f}$

* Anggaplah Anda membaca kata-kata dalam sebuah teks, dan berencana menggunakan LSTM untuk melacak struktur tata bahasa, seperti apakah subjeknya tunggal ("anak anjing") atau jamak ("anak anjing").
* Jika subjek mengubah statenya (dari kata tunggal menjadi kata jamak), memori state sebelumnya menjadi usang, sehingga Anda akan "melupakan" state usang tersebut.
* "Forget Gate" adalah tensor yang berisi nilai antara 0 dan 1.
* Jika unit di forget gate memiliki nilai mendekati 0, LSTM akan "melupakan" status yang disimpan di unit terkait dari status sel sebelumnya.
* Jika unit di forget gate memiliki nilai mendekati 1, sebagian besar LSTM akan mengingat nilai terkait dalam keadaan tersimpan.

##### Persamaan

$$\mathbf{\Gamma}_f^{\langle t \rangle} = \sigma(\mathbf{W}_f[\mathbf{a}^{\langle t-1 \rangle}, \mathbf{x}^{\langle t \rangle}] + \mathbf{b}_f)\tag{1} $$


##### Penjelasan persamaan:
* $\mathbf{W_{f}}$ berisi bobot yang mengatur perilaku forget gate.
* $[a^{\langle t-1 \rangle}$ hidden state langkah pindah sebelumnya dan input $x^{\langle t \rangle}]$ langkah pindah saat ini digabungkan dan dikalikan dengan $\mathbf{W_{f}}$.
* Fungsi sigmoid digunakan untuk membuat setiap nilai tensor gate $\mathbf{\Gamma}_f^{\langle t \rangle}$ berkisar dari 0 hingga 1.
* Forget gate $\mathbf{\Gamma}_f^{\langle t \rangle}$ mempunyai dimensi yang sama dengan state sel $c^{\langle t-1 \rangle}$ sebelumnya.
* Artinya keduanya dapat dikalikan berdasarkan elemen.
* Mengalikan tensor $\mathbf{\Gamma}_f^{\langle t \rangle} * \mathbf{c}^{\langle t-1 \rangle}$ seperti menerapkan penutup pada state sel sebelumnya.
* Jika nilai tunggal di $\mathbf{\Gamma}_f^{\langle t \rangle}$ adalah 0 atau mendekati 0, maka hasil kali mendekati 0.
* Ini menjaga informasi yang disimpan dalam unit terkait di $\mathbf{c}^{\langle t-1 \rangle}$ agar tidak diingat untuk langkah waktu berikutnya.
* Demikian pula, jika salah satu nilai mendekati 1, hasil perkaliannya mendekati nilai asli pada state sel sebelumnya.
* LSTM akan menyimpan informasi dari unit $\mathbf{c}^{\langle t-1 \rangle}$ yang sesuai, untuk digunakan pada langkah pindah berikutnya.
    
##### Nama variabel dalam code
Nama variabel dalam code mirip dengan persamaan, dengan sedikit perbedaan.
* `Wf`: forget gate weight $\mathbf{W}_{f}$
*`bf`: forget gate bias $\mathbf{b}_{f}$
* `ft`: forget gate $\Gamma_f^{\langle t \rangle}$

#### Nilai kandidat $\tilde{\mathbf{c}}^{\langle t \rangle}$
* Nilai kandidat adalah tensor yang berisi informasi dari langkah pindah saat ini yang **mungkin** disimpan dalam state sel saat ini $\mathbf{c}^{\langle t \rangle}$.
* Bagian dari nilai kandidat yang diteruskan bergantung pada update gate.
* Nilai kandidat adalah tensor yang berisi nilai yang berkisar dari -1 hingga 1.
* Tanda tilde "~" digunakan untuk membedakan kandidat $\tilde{\mathbf{c}}^{\langle t \rangle}$ dari state sel $\mathbf{c}^{\langle t \rangle}$.

##### Persamaan
$$\mathbf{\tilde{c}}^{\langle t \rangle} = \tanh\left( \mathbf{W}_{c} [\mathbf{a}^{\langle t - 1 \rangle}, \mathbf{x}^{\langle t \rangle}] + \mathbf{b}_{c} \right) \tag{3}$$


##### Penjelasan persamaan
* Fungsi *tanh* menghasilkan nilai antara -1 dan 1.


##### Nama variabel dalam kode
* `cct`: nilai kandidat $\mathbf{\tilde{c}}^{\langle t \rangle}$

#### Update Gate $\mathbf{\Gamma}_{i}$

* Anda menggunakan update gate untuk memutuskan aspek kandidat $\tilde{\mathbf{c}}^{\langle t \rangle}$ mana yang akan ditambahkan ke state sel $c^{\langle t \rangle}$.
* Update gate memutuskan bagian mana dari tensor "kandidat" $\tilde{\mathbf{c}}^{\langle t \rangle}$ yang diteruskan ke state sel $\mathbf{c}^{\langle t \rangle}$.
* Update gate adalah tensor yang berisi nilai antara 0 dan 1.
* Ketika unit di update gate mendekati 1, ini memungkinkan nilai kandidat $\tilde{\mathbf{c}}^{\langle t \rangle}$ diteruskan ke hidden state $\mathbf{c}^{\langle t \rangle}$
* Ketika unit di update gate mendekati 0, hal ini mencegah nilai terkait dalam kandidat diteruskan ke hidden state.
* Perhatikan bahwa subscript "i" digunakan dan bukan "u", untuk mengikuti konvensi yang digunakan dalam literatur.

##### Persamaan

$$\mathbf{\Gamma}_i^{\langle t \rangle} = \sigma(\mathbf{W}_i[a^{\langle t-1 \rangle}, \mathbf{x}^{\langle t \rangle}] + \mathbf{b}_i)\tag{2} $$ 

##### Penjelasan persamaan

* Mirip dengan forget gate, di sini $\mathbf{\Gamma}_i^{\langle t \rangle}$, sigmoid menghasilkan nilai antara 0 dan 1.
* Update gate dikalikan berdasarkan elemen dengan kandidat, dan hasil kali ini ($\mathbf{\Gamma}_{i}^{\langle t \rangle} * \tilde{c}^{\langle t \rangle}$) digunakan dalam menentukan state sel $\mathbf{c}^{\langle t \rangle}$.

##### Nama variabel dalam code (Harap diperhatikan bahwa nama variabel berbeda dengan persamaan)
Dalam code tersebut, Anda akan menggunakan nama variabel yang ditemukan dalam literatur akademis. Variabel-variabel ini tidak menggunakan "u" untuk menunjukkan "update".
* `Wi` adalah update gate weight $\mathbf{W}_i$ (bukan "Wu")
* `bi` adalah update gate bias $\mathbf{b}_i$ (bukan "bu")
* `itu` adalah update gate $\mathbf{\Gamma}_i^{\langle t \rangle}$ (bukan "ut")

#### Cell state $\mathbf{c}^{\langle t \rangle}$

* cell state adalah "memori" yang diteruskan ke langkah waktu mendatang.
* state cell baru $\mathbf{c}^{\langle t \rangle}$ adalah kombinasi dari state cell sebelumnya dan nilai kandidat.

##### Persamaan

$$ \mathbf{c}^{\langle t \rangle} = \mathbf{\Gamma}_f^{\langle t \rangle}* \mathbf{c}^{\langle t-1 \rangle} + \mathbf{\Gamma}_{i}^{\langle t \rangle} *\mathbf{\tilde{c}}^{\langle t \rangle} \tag{4} $$

##### Penjelasan persamaan
*Keadaan sel sebelumnya $\mathbf{c}^{\langle t-1 \rangle}$ disesuaikan (weighted) oleh forget gate $\mathbf{\Gamma}_{f}^{\langle t \rangle}$* dan nilai kandidat $\tilde{\mathbf{c}}^{\langle t \rangle}$, disesuaikan (weighted) dengan update gate $\mathbf{\Gamma}_{i}^{\langle t \rangle}$

##### Nama dan bentuk variabel dalam kode
* `c`: cell state, termasuk semua langkah pindah, bentuk $\mathbf{c}$ $(n_{a}, m, T_x)$
* `c_next`: cell state baru (berikutnya), bentuk $\mathbf{c}^{\langle t \rangle}$ $(n_{a}, m)$
* `c_prev`: cell_state sebelumnya, $\mathbf{c}^{\langle t-1 \rangle}$, bentuk $(n_{a}, m)$

#### Output gate $\mathbf{\Gamma}_{o}$

* output gate memutuskan apa yang dikirim sebagai prediksi (output) dari suatu langkah pindah.
* output gate sama seperti gate lainnya, di dalamnya berisi nilai yang berkisar dari 0 hingga 1.

##### Persamaan

$$ \mathbf{\Gamma}_o^{\langle t \rangle}=  \sigma(\mathbf{W}_o[\mathbf{a}^{\langle t-1 \rangle}, \mathbf{x}^{\langle t \rangle}] + \mathbf{b}_{o})\tag{5}$$ 

##### Penjelasan persamaan
* output gate ditentukan oleh hidden state sebelumnya $\mathbf{a}^{\langle t-1 \rangle}$ dan input saat ini $\mathbf{x}^{\langle t \rangle}$
* Sigmoid membuat gate berkisar dari 0 hingga 1.


##### Nama variabel dalam kode
* `Wo`: output gate weight, $\mathbf{W_o}$
* `bo`: output gate bias, $\mathbf{b_o}$
* `ot`: output gate, $\mathbf{\Gamma}_{o}^{\langle t \rangle}$

#### Hidden State $\mathbf{a}^{\langle t \rangle}$

* Hidden state diteruskan ke langkah pindah sel LSTM berikutnya.
* Digunakan untuk menentukan tiga gate ($\mathbf{\Gamma}_{f}, \mathbf{\Gamma}_{u}, \mathbf{\Gamma}_{o}$) langkah pindah berikutnya.
* Hidden state juga digunakan untuk prediksi $y^{\langle t \rangle}$.

##### Persamaan

$$ \mathbf{a}^{\langle t \rangle} = \mathbf{\Gamma}_o^{\langle t \rangle} * \tanh(\mathbf{c}^{\langle t \rangle})\tag{6} $$

##### Penjelasan persamaan
* Hidden state $\mathbf{a}^{\langle t \rangle}$ ditentukan oleh state sel $\mathbf{c}^{\langle t \rangle}$ yang dikombinasikan dengan output gate $\mathbf{\Gamma}_{o}$.
* State sel diteruskan melalui fungsi `tanh` untuk mengubah skala nilai antara -1 dan 1.
* Output gate bertindak seperti "mask" yang mempertahankan nilai-nilai $\tanh(\mathbf{c}^{\langle t \rangle})$ atau menjaga nilai-nilai tersebut agar tidak dimasukkan dalam hidden state $\mathbf{a}^{\langle t \rangle}$

##### Nama dan bentuk variabel dalam code
* `a`: hidden state, termasuk langkah pindahnya. $\mathbf{a}$ memiliki bentuk $(n_{a}, m, T_{x})$
* `a_prev`: hidden state dari langkah pindah sebelumnya. $\mathbf{a}^{\langle t-1 \rangle}$ memiliki bentuk $(n_{a}, m)$
* `a_next`: hidden state untuk langkah pindah berikutnya. $\mathbf{a}^{\langle t \rangle}$ memiliki bentuk $(n_{a}, m)$

#### Prediction $\mathbf{y}^{\langle t \rangle}_{pred}$
* The prediction in this use case is a classification, so you'll use a softmax.

The equation is:
$$\mathbf{y}^{\langle t \rangle}_{pred} = \textrm{softmax}(\mathbf{W}_{y} \mathbf{a}^{\langle t \rangle} + \mathbf{b}_{y})$$

##### Variable names and shapes in the code
* `y_pred`: prediction, including all time steps. $\mathbf{y}_{pred}$ has shape $(n_{y}, m, T_{x})$.  Note that $(T_{y} = T_{x})$ for this example.
* `yt_pred`: prediction for the current time step $t$. $\mathbf{y}^{\langle t \rangle}_{pred}$ has shape $(n_{y}, m)$

#### Prediksi $\mathbf{y}^{\langle t \rangle}_{pred}$
* Prediksi dalam use case ini adalah klasifikasi, jadi Anda akan menggunakan softmax.

Persamaannya adalah:
$$\mathbf{y}^{\langle t \rangle}_{pred} = \textrm{softmax}(\mathbf{W}_{y} \mathbf{a}^{\langle t \rangle} + \mathbf{b}_{y})$$

##### Nama dan bentuk variabel dalam kode
* `y_pred`: prediksi, termasuk semua langkah pindah. $\mathbf{y}_{pred}$ memiliki bentuk $(n_{y}, m, T_{x})$. Perhatikan bahwa $(T_{y} = T_{x})$ untuk contoh ini.
* `yt_pred`: prediksi untuk langkah pindah saat ini $t$. $\mathbf{y}^{\langle t \rangle}_{pred}$ memiliki bentuk $(n_{y}, m)$

<a name='2-1'></a>
### LSTM Cell

<a name='ex-3'></a>
### Tugas 3 - lstm_cell_forward

Implementasikan sel LSTM yang dijelaskan pada Gambar 4.

**Petunjuk**:
1. Gabungkan hidden state $a^{\langle t-1 \rangle}$ dan input $x^{\langle t \rangle}$ ke dalam satu matriks:

$$concat = \begin{bmatrix} a^{\langle t-1 \rangle} \\ x^{\langle t \rangle} \end{bmatrix}$$  

2. Hitung semua rumus (1 sampai 6) untuk gate, hidden state, dan cell state.
3. Hitung prediksi $y^{\langle t \rangle}$.

#### Petunjuk Tambahan
* Anda dapat menggunakan [numpy.concatenate](https://docs.scipy.org/doc/numpy/reference/generated/numpy.concatenate.html). Periksa nilai mana yang akan digunakan untuk parameter `axis`.
* Fungsi `sigmoid()` dan `softmax` diimpor dari `rnn_utils.py`.
* Beberapa dokumen untuk [numpy.tanh](https://docs.scipy.org/doc/numpy/reference/generated/numpy.tanh.html)
* Gunakan [numpy.dot](https://docs.scipy.org/doc/numpy/reference/generated/numpy.dot.html) untuk perkalian matriks.
* Perhatikan bahwa nama variabel `Wi`, `bi` mengacu pada bobot dan bias **update gate**. Tidak ada variabel bernama "Wu" atau "bu" dalam fungsi ini.

In [6]:
# UNQ_C3 (UNIQUE CELL IDENTIFIER, DO NOT EDIT)
# GRADED FUNCTION: lstm_cell_forward

def lstm_cell_forward(xt, a_prev, c_prev, parameters):
    """
    Implement a single forward step of the LSTM-cell as described in Figure (4)

    Arguments:
    xt -- your input data at timestep "t", numpy array of shape (n_x, m).
    a_prev -- Hidden state at timestep "t-1", numpy array of shape (n_a, m)
    c_prev -- Memory state at timestep "t-1", numpy array of shape (n_a, m)
    parameters -- python dictionary containing:
                        Wf -- Weight matrix of the forget gate, numpy array of shape (n_a, n_a + n_x)
                        bf -- Bias of the forget gate, numpy array of shape (n_a, 1)
                        Wi -- Weight matrix of the update gate, numpy array of shape (n_a, n_a + n_x)
                        bi -- Bias of the update gate, numpy array of shape (n_a, 1)
                        Wc -- Weight matrix of the first "tanh", numpy array of shape (n_a, n_a + n_x)
                        bc --  Bias of the first "tanh", numpy array of shape (n_a, 1)
                        Wo -- Weight matrix of the output gate, numpy array of shape (n_a, n_a + n_x)
                        bo --  Bias of the output gate, numpy array of shape (n_a, 1)
                        Wy -- Weight matrix relating the hidden-state to the output, numpy array of shape (n_y, n_a)
                        by -- Bias relating the hidden-state to the output, numpy array of shape (n_y, 1)
                        
    Returns:
    a_next -- next hidden state, of shape (n_a, m)
    c_next -- next memory state, of shape (n_a, m)
    yt_pred -- prediction at timestep "t", numpy array of shape (n_y, m)
    cache -- tuple of values needed for the backward pass, contains (a_next, c_next, a_prev, c_prev, xt, parameters)
    
    Note: ft/it/ot stand for the forget/update/output gates, cct stands for the candidate value (c tilde),
          c stands for the cell state (memory)
    """

    # Retrieve parameters from "parameters"
    Wf = parameters["Wf"] # forget gate weight
    bf = parameters["bf"]
    Wi = parameters["Wi"] # update gate weight (notice the variable name)
    bi = parameters["bi"] # (notice the variable name)
    Wc = parameters["Wc"] # candidate value weight
    bc = parameters["bc"]
    Wo = parameters["Wo"] # output gate weight
    bo = parameters["bo"]
    Wy = parameters["Wy"] # prediction weight
    by = parameters["by"]
    
    # Retrieve dimensions from shapes of xt and Wy
    n_x, m = xt.shape
    n_y, n_a = Wy.shape

    ### START CODE HERE ###
    # Concatenate a_prev and xt (≈1 line)
    concat = np.concatenate((a_prev, xt), axis=0)

    # Compute values for ft, it, cct, c_next, ot, a_next using the formulas given figure (4) (≈6 lines)
    ft = sigmoid(np.dot(Wf, concat) + bf)
    it = sigmoid(np.dot(Wi, concat) + bi)
    cct = np.tanh(np.dot(Wc, concat) + bc)
    c_next = ft * c_prev + it * cct
    ot = sigmoid(np.dot(Wo, concat) + bo)
    a_next = ot * np.tanh(c_next)
    
    # Compute prediction of the LSTM cell (≈1 line)
    yt_pred = softmax(np.dot(Wy, a_next) + by)
    ### END CODE HERE ###

    # store values needed for backward propagation in cache
    cache = (a_next, c_next, a_prev, c_prev, ft, it, cct, ot, xt, parameters)

    return a_next, c_next, yt_pred, cache

In [7]:
np.random.seed(1)
xt_tmp = np.random.randn(3, 10)
a_prev_tmp = np.random.randn(5, 10)
c_prev_tmp = np.random.randn(5, 10)
parameters_tmp = {}
parameters_tmp['Wf'] = np.random.randn(5, 5 + 3)
parameters_tmp['bf'] = np.random.randn(5, 1)
parameters_tmp['Wi'] = np.random.randn(5, 5 + 3)
parameters_tmp['bi'] = np.random.randn(5, 1)
parameters_tmp['Wo'] = np.random.randn(5, 5 + 3)
parameters_tmp['bo'] = np.random.randn(5, 1)
parameters_tmp['Wc'] = np.random.randn(5, 5 + 3)
parameters_tmp['bc'] = np.random.randn(5, 1)
parameters_tmp['Wy'] = np.random.randn(2, 5)
parameters_tmp['by'] = np.random.randn(2, 1)

a_next_tmp, c_next_tmp, yt_tmp, cache_tmp = lstm_cell_forward(xt_tmp, a_prev_tmp, c_prev_tmp, parameters_tmp)

print("a_next[4] = \n", a_next_tmp[4])
print("a_next.shape = ", a_next_tmp.shape)
print("c_next[2] = \n", c_next_tmp[2])
print("c_next.shape = ", c_next_tmp.shape)
print("yt[1] =", yt_tmp[1])
print("yt.shape = ", yt_tmp.shape)
print("cache[1][3] =\n", cache_tmp[1][3])
print("len(cache) = ", len(cache_tmp))

# UNIT TEST
lstm_cell_forward_test(lstm_cell_forward)

a_next[4] = 
 [-0.66408471  0.0036921   0.02088357  0.22834167 -0.85575339  0.00138482
  0.76566531  0.34631421 -0.00215674  0.43827275]
a_next.shape =  (5, 10)
c_next[2] = 
 [ 0.63267805  1.00570849  0.35504474  0.20690913 -1.64566718  0.11832942
  0.76449811 -0.0981561  -0.74348425 -0.26810932]
c_next.shape =  (5, 10)
yt[1] = [0.79913913 0.15986619 0.22412122 0.15606108 0.97057211 0.31146381
 0.00943007 0.12666353 0.39380172 0.07828381]
yt.shape =  (2, 10)
cache[1][3] =
 [-0.16263996  1.03729328  0.72938082 -0.54101719  0.02752074 -0.30821874
  0.07651101 -1.03752894  1.41219977 -0.37647422]
len(cache) =  10
All tests passed


**Expected Output**:

```Python
a_next[4] = 
 [-0.66408471  0.0036921   0.02088357  0.22834167 -0.85575339  0.00138482
  0.76566531  0.34631421 -0.00215674  0.43827275]
a_next.shape =  (5, 10)
c_next[2] = 
 [ 0.63267805  1.00570849  0.35504474  0.20690913 -1.64566718  0.11832942
  0.76449811 -0.0981561  -0.74348425 -0.26810932]
c_next.shape =  (5, 10)
yt[1] = [ 0.79913913  0.15986619  0.22412122  0.15606108  0.97057211  0.31146381
  0.00943007  0.12666353  0.39380172  0.07828381]
yt.shape =  (2, 10)
cache[1][3] =
 [-0.16263996  1.03729328  0.72938082 -0.54101719  0.02752074 -0.30821874
  0.07651101 -1.03752894  1.41219977 -0.37647422]
len(cache) =  10
```

<a name='2-2'></a>
### Forward Pass untuk LSTM

Setelah mengimplementasikan satu langkah LSTM, Anda dapat mengulanginya menggunakan perulangan for untuk memproses rangkaian input $T_x$.

<img src="images/LSTM_rnn.png" style="width:500;height:300px;">
<caption><center><font color='purple'><b>Gambar 5</b>: LSTM dalam beberapa langkah pindah. </center></caption>

<a name='ex-4'></a>
### Tugas 4 - lstm_forward
    
Implementasi `lstm_forward()` untuk menjalankan LSTM melalui langkah pindah $T_x$.

**Petunjuk**
* Dapatkan dimensi $n_x, n_a, n_y, m, T_x$ dari bentuk variabel: `x` dan `parameter`* Inisiasi tensor 3D $a$, $c$, dan $y$
    - $a$: hidden state, bentuk $(n_{a}, m, T_{x})$
    - $c$: cell state, bentuk $(n_{a}, m, T_{x})$
    - $y$: prediksi, bentuk $(n_{y}, m, T_{x})$ (Perhatikan bahwa $T_{y} = T_{x}$ dalam contoh ini)
    - **Catatan** Menyetel satu variabel sama dengan variabel lainnya adalah "copy by reference". Dengan kata lain, jangan lakukan `c = a', jika tidak, kedua variabel ini menunjuk ke variabel dasar yang sama.
*Inisiasi tensor 2D $a^{\langle t \rangle}$
    - $a^{\langle t \rangle}$ menyimpan hidden state untuk langkah pindah $t$. Nama variabelnya adalah `a_next`.
    - $a^{\langle 0 \rangle}$, hidden state awal pada langkah waktu 0, diteruskan saat memanggil fungsi. Nama variabelnya adalah `a0`.
    - $a^{\langle t \rangle}$ dan $a^{\langle 0 \rangle}$ mewakili satu langkah pindah, sehingga keduanya memiliki bentuk $(n_{a}, m)$
    - Inisiasi $a^{\langle t \rangle}$ dengan mengaturnya ke hidden state awal ($a^{\langle 0 \rangle}$) yang diteruskan ke fungsi.
* Inisiasi $c^{\langle t \rangle}$ dengan nol.
    - Nama variabelnya adalah `c_next`
    - $c^{\langle t \rangle}$ mewakili satu langkah pindah, jadi bentuknya adalah $(n_{a}, m)$
    - **Catatan**: buat `c_next` sebagai variabelnya sendiri dengan lokasinya sendiri di memori. Jangan inisiasi sebagai bagian dari tensor 3D $c$. Dengan kata lain, **jangan** lakukan `c_next = c[:,:,0]`.
* Untuk setiap langkah waktu, lakukan hal berikut:
    - Dari tensor 3D $x$, dapatkan irisan 2D $x^{\langle t \rangle}$ pada langkah waktu $t$
    - Panggil fungsi `lstm_cell_forward` yang Anda tentukan sebelumnya, untuk mendapatkan hidden state, cell state, prediksi, dan cache
    - Simpan hidden state, cell state, dan prediksi (tensor 2D) di dalam tensor 3D
    - Tambahkan cache ke daftar cache

In [8]:
# UNQ_C4 (UNIQUE CELL IDENTIFIER, DO NOT EDIT)
# GRADED FUNCTION: lstm_forward

def lstm_forward(x, a0, parameters):
    """
    Implement the forward propagation of the recurrent neural network using an LSTM-cell described in Figure (4).

    Arguments:
    x -- Input data for every time-step, of shape (n_x, m, T_x).
    a0 -- Initial hidden state, of shape (n_a, m)
    parameters -- python dictionary containing:
                        Wf -- Weight matrix of the forget gate, numpy array of shape (n_a, n_a + n_x)
                        bf -- Bias of the forget gate, numpy array of shape (n_a, 1)
                        Wi -- Weight matrix of the update gate, numpy array of shape (n_a, n_a + n_x)
                        bi -- Bias of the update gate, numpy array of shape (n_a, 1)
                        Wc -- Weight matrix of the first "tanh", numpy array of shape (n_a, n_a + n_x)
                        bc -- Bias of the first "tanh", numpy array of shape (n_a, 1)
                        Wo -- Weight matrix of the output gate, numpy array of shape (n_a, n_a + n_x)
                        bo -- Bias of the output gate, numpy array of shape (n_a, 1)
                        Wy -- Weight matrix relating the hidden-state to the output, numpy array of shape (n_y, n_a)
                        by -- Bias relating the hidden-state to the output, numpy array of shape (n_y, 1)
                        
    Returns:
    a -- Hidden states for every time-step, numpy array of shape (n_a, m, T_x)
    y -- Predictions for every time-step, numpy array of shape (n_y, m, T_x)
    c -- The value of the cell state, numpy array of shape (n_a, m, T_x)
    caches -- tuple of values needed for the backward pass, contains (list of all the caches, x)
    """

    # Initialize "caches", which will track the list of all the caches
    caches = []
    
    ### START CODE HERE ###
    Wy = parameters['Wy'] # saving parameters['Wy'] in a local variable in case students use Wy instead of parameters['Wy']
    # Retrieve dimensions from shapes of x and parameters['Wy'] (≈2 lines)
    n_x, m, T_x = x.shape
    n_y, n_a = parameters["Wy"].shape
    
    # initialize "a", "c" and "y" with zeros (≈3 lines)
    a = np.zeros((n_a, m, T_x))
    c = np.zeros((n_a, m, T_x))
    y = np.zeros((n_y, m, T_x))
    
    # Initialize a_next and c_next (≈2 lines)
    a_next = a0
    c_next = np.zeros(a_next.shape)
    
    # loop over all time-steps
    for t in range(T_x):
        # Get the 2D slice 'xt' from the 3D input 'x' at time step 't'
        xt = x[:,:,t]
        # Update next hidden state, next memory state, compute the prediction, get the cache (≈1 line)
        a_next, c_next, yt, cache = lstm_cell_forward(xt, a_next, c_next, parameters)
        # Save the value of the new "next" hidden state in a (≈1 line)
        a[:,:,t] = a_next
        # Save the value of the next cell state (≈1 line)
        c[:,:,t]  = c_next
        # Save the value of the prediction in y (≈1 line)
        y[:,:,t] = yt
        # Append the cache into caches (≈1 line)
        caches.append(cache)
        
    ### END CODE HERE ###
    
    # store values needed for backward propagation in cache
    caches = (caches, x)

    return a, y, c, caches

In [9]:
np.random.seed(1)
x_tmp = np.random.randn(3, 10, 7)
a0_tmp = np.random.randn(5, 10)
parameters_tmp = {}
parameters_tmp['Wf'] = np.random.randn(5, 5 + 3)
parameters_tmp['bf'] = np.random.randn(5, 1)
parameters_tmp['Wi'] = np.random.randn(5, 5 + 3)
parameters_tmp['bi']= np.random.randn(5, 1)
parameters_tmp['Wo'] = np.random.randn(5, 5 + 3)
parameters_tmp['bo'] = np.random.randn(5, 1)
parameters_tmp['Wc'] = np.random.randn(5, 5 + 3)
parameters_tmp['bc'] = np.random.randn(5, 1)
parameters_tmp['Wy'] = np.random.randn(2, 5)
parameters_tmp['by'] = np.random.randn(2, 1)

a_tmp, y_tmp, c_tmp, caches_tmp = lstm_forward(x_tmp, a0_tmp, parameters_tmp)
print("a[4][3][6] = ", a_tmp[4][3][6])
print("a.shape = ", a_tmp.shape)
print("y[1][4][3] =", y_tmp[1][4][3])
print("y.shape = ", y_tmp.shape)
print("caches[1][1][1] =\n", caches_tmp[1][1][1])
print("c[1][2][1]", c_tmp[1][2][1])
print("len(caches) = ", len(caches_tmp))

# UNIT TEST    
lstm_forward_test(lstm_forward)

a[4][3][6] =  0.1721177675329167
a.shape =  (5, 10, 7)
y[1][4][3] = 0.9508734618501101
y.shape =  (2, 10, 7)
caches[1][1][1] =
 [ 0.82797464  0.23009474  0.76201118 -0.22232814 -0.20075807  0.18656139
  0.41005165]
c[1][2][1] -0.8555449167181983
len(caches) =  2
All tests passed


**Expected Output**:

```Python
a[4][3][6] =  0.172117767533
a.shape =  (5, 10, 7)
y[1][4][3] = 0.95087346185
y.shape =  (2, 10, 7)
caches[1][1][1] =
 [ 0.82797464  0.23009474  0.76201118 -0.22232814 -0.20075807  0.18656139
  0.41005165]
c[1][2][1] -0.855544916718
len(caches) =  2
```